 # Physics 226 Fall 2022 Problem Set 5 Due Friday Oct 30

## Question 1: Fitting a Signal in the Presence of Background (25 points)

### Learning objectives
In this question you will:

- Gain experience in performing $\chi^2$ fits to histogrammed data; explore how the statistical significance of a signal depends on the number of signal events and the signal-to-background ratio


### 1a. 

Physicists often fit for signals in the presence of background.  In such cases, the significance of the fitted signal depends not only on the number of signal events but also on the amount of background and our ability to statistically separate the two.  In this problem, we will explore fitting signal and background for a very simple case:  The signal is a Gaussian peak centered at $x=10$ with a width $\sigma=1$ and the background is uniformly distributed between $x=0$ and $x=20$.  The code below generates fake data, allowing you to change both the number of events in the signal and the ratio of signal-to-background.  To make sure that our definition of background does not depend on the fit range, the code below defines the signal-to-background ratio as the ratio number of signal events to the number of background events in a $\pm 2 \sigma$ window around the signal peak.  Here is the code you will use to generate the fake data:

In [2]:
# Write import math
import numpy as np
import random
import matplotlib.pyplot as plt
from scipy.optimize import minimize

def makeData( SignalToBackground, nSigEvents, seed=12345):
    """Generates a dataset consisting of nEvents some of which are signal (a Gaussian with a width of 1 centered
    at x=5) and the rest of which is background
    Definition of SignalToBackground:  The ratio of the number of signal events within 2 sigma of the peak to the number
    of background events in that same x-range  (this definition is just a choice)
    
    Parameters
    ==========
     SignalToBackground : float
     Ratio of the number of signal events within 2 sigma of the peak to the number
    of background events in that same x-range  (this definition is just a choice) 
      
    nSigEvents : int
      total number of signal events generated (NOT the number in +- 2 sigma)
      
    seed : int
      seed for the random number generator
      
    Returns
    =======
    data : array
      the measurements of x
    """
    fracOutsideTwoSigma = 4.55e-2
    # 1-fracOutsideTwoSigma is the fraction of the Signal Events within +-2 sigma
    # To get the total number of background events, find the number in +-2 sigma
    # which is 4 units of x and then since the background is flat from 0 to 20
    # multiply by 20/4=5
    
    nBackground = 5*nSigEvents*(1-fracOutsideTwoSigma)/SignalToBackground
    nEvents = nSigEvents+nBackground
    fSig = nSigEvents/nEvents
    
    # Make an array to hold the data (ie the x measurements)
    data = []
    
    # set the random seed.  This will allow us to reproduce the results if we run again
    np.random.seed(seed)
 
    # Retrieve nEvents random numbers that will be used to pick which events are signal
    # and which are background
    n = int(nEvents)
    tests = np.random.uniform(0,1,n)
    bck = np.random.uniform(0,20,n)
    sig = np.random.normal(10,1,n)
    
    count = 0
    for test in tests:
        if(test<fSig):
            data.append(sig[count])
        else:
            data.append(bck[count])
        count+=1
    
    # Loop over the events and pick either signal or background and draw from the appropriate 
    # pdf in each case
    
    # return the data to the user
    return data
    

Below is a simple test to show you how to use this code.  Remember if you run this code multiple times you should change the seed each time (for example, you could increment the seed each time you call the function)

In [3]:
mydata = []
#  make 10 signal events with a signalToBackground of 1 using random seed 123
mydata=makeData(1,20,123)
print("number of total data events: ",len(mydata))
print("data: ",mydata)

number of total data events:  115
data:  [11.047401505881462, 11.57102936217666, 10.430661187946646, 0.05376129148641384, 19.7669083856564, 18.106831513232198, 4.15271722392649, 5.849788255848496, 10.40020306144967, 18.038227453213413, 19.672617698234465, 5.150841283081662, 11.287180858495633, 10.579689779144333, 7.887401079055496, 14.621460716891141, 3.221380288584297, 12.013971356671798, 17.317289166065294, 19.67043218407111, 1.5873158075603144, 8.566945494018984, 4.0908571909285545, 9.012729810374696, 10.95527145257708, 1.8665342073964153, 5.937215509613589, 18.55168480304295, 11.380074628603907, 9.148239950472238, 8.762646787312146, 14.837243036840746, 0.9715806568853758, 14.17394790885492, 16.784866956101673, 3.3187576841390776, 15.619958759999147, 5.730732334582038, 6.129395066591146, 13.30522930699366, 2.227843432154315, 9.800887617242747, 17.757135853524453, 13.926225364708127, 8.806557533308181, 8.764287687544494, 15.30192190478613, 11.312840024517657, 1.6980832638363519, 11.6

Generate a sample of 1000 signal events with signalToBackground=0.5 and make a histogram of your results. Make sure that the number of bins in your histogram is small enough that you have at least 10 entries per bin (so that it is reasonable to do a binned fit to the histogram). To make life a bit easier, here is function you can use to make your histograms.  You are of course free to write your own function and not use this code.

In [4]:
#Import the pyplot module of matplotlib as "plt"
import matplotlib.pyplot as plt


#Makes a histogram filled with the random numbers we generate
def plot_histogram(samples,xtitle,ytitle, title, nbins, limits):
   
    #Plot the histogram of the sampled data with nbins and a nice color
    n, bins, patches =plt.hist(samples, bins=nbins, range=limits, color=(0,0.7,0.9))  #Set the color using (r,g,b) values or
                                                                  #  use a built-in matplotlib color""" 
    bincenters = 0.5*(bins[1:]+bins[:-1])
    errs = np.sqrt(n)

    plt.errorbar(bincenters, n, yerr=errs, fmt='none')
    #Add some axis labels and a descriptive title
    plt.xlabel(xtitle)
    plt.ylabel(ytitle)

    #Get rid of the extra white space on the left/right edges (you can delete these two lines without a problem)
    xmin, xmax, ymin, ymax = plt.axis()
    plt.axis([limits[0],limits[1],ymin,ymax])

    #Not necessarily needed in a Jupyter notebook, but it doesn't hurt
    plt.show()
    return n, bins, patches

In [5]:
#Write your answer here

### 1b. 

Pretend this is real data.  Your goal is to find the best estimate of how many events are in a Gaussian peak with unknown mean and sigma and what the uncertainty on this estimate is.  In your fit, you can make the assumption that the background is flat (a zeroth order polynomial) but that you don't have a prediction for the background rate.  Use your favorite minimization package to fit the data.  Deterimine the best estimate of the number of events in signal and the uncertainty on that estimate.  (remember, that you must let the position and width of the Gaussian and the size of the background vary in your fit).

Hint: For examples of how to perform a least squared fit of a function to data see:
- https://github.com/berkeley-physics/Python-Tutorials/blob/master/3%20-%20Specific%20topics/Fitting.ipynb
- https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.curve_fit.html

In [6]:
#Write your answer here

### 1c. 

What is the $\chi^2$ per degree of freedom for your fit?  What does this number tell you about how well your fit describes the data?

In [7]:
#Write your answer here

### 1d. 

If a signal of size $N_S^{meas}$ has a fitted uncertainty $\sigma_S$, the signficance of the measurement is defined to be
$$
S^{meas} \equiv \frac{N_S^{meas}}{\sigma_S}
$$
When the size of the data sample is large enough that Gaussian uncertainties are appropriate, a rule of thumb can be used to give a crude estimate of the expected significance $S^{expected}$ of the measurement. This predicted signficance depends on the number of signal events ($N_{S}$) and is the number of background events populating the region **beneath the signal** ($N_B$):
$$
S^{expected}  \approx  \frac{N_S}{N_S+N_B}
$$

Repeat the above exercise changing both the number of events in your signal and the signalToBackground ratio.  Plot the values of the measured significance  $S^{meas}$ 
obtained from your fits as a function of $\frac{N_S}{\sqrt{N_S+N_B}}$.  How do your results compare to the simple rule of thumb quoted above?

In [8]:
#Write your answer here

## Question 2: Isospin: $\pi N$ scattering (15 Points)

### Learning objectives
In this question you will:
- Review how conservation of isospin in the strong interactions can be used to related scattering rates for different processes related by isospin
- Apply this concept to pion nucleon scattering

Consider the scattering of a pion with a nucleon.  Since the pion has $I=1$ and the nucleon has $I=\frac{1}{2}$, there are two possible values of total isospin for this scattering:  $I=\frac{3}{2}$ and $I=\frac{1}{2}$.  Isospin conservation in the strong interactions means that the matrix element $H_{if} = \left < f \right | H_{int} \left  | i \right >$ (where $i$ and $f$ are the initial and final states) depends only on $I$ and not on $I_z$.  

There are six possible *elastic scattering* processes:

\begin{eqnarray*}
\pi^{+} p  & \rightarrow & \pi^{+} p \\
\pi^{0} p  & \rightarrow & \pi^{0} p \\
\pi^{-} p  & \rightarrow & \pi^{-} p \\
\pi^{+} n  & \rightarrow & \pi^{+} n \\
\pi^{0} n  & \rightarrow & \pi^{0} n \\
\pi^{-} n  & \rightarrow & \pi^{-} n \\
\end{eqnarray*}

And four possible *charge exchange* processes:

\begin{eqnarray*}
\pi^{+} n & \rightarrow & \pi^{-} p \\
\pi^{0} p & \rightarrow & \pi^{+} n \\
\pi^{-} p & \rightarrow & \pi^{0} n \\
\pi^{0} n & \rightarrow & \pi^{-} p \\
\end{eqnarray*}


Since all 10 of these processes have the same phase space terms, the relative rates for the processes can be calculated in terms of only two independent terms: $M_{\frac{1}{2}} = \left < I=\frac{1}{2}\right | H_{int} \left | I= \frac{1}{2}\right >$ and $M_{\frac{3}{2}} = \left < I=\frac{3}{2}\right | H_{int} \left | I= \frac{3}{2}\right >$

### 2a.

For the state $\pi^+ p$ we can describe the isospin of the state in the uncoupled and coupled bases:
$$
\left |m_1=+1, m_2=\frac{1}{2} \right > = \left |I=\frac{3}{2}, m=\frac{3}{2} \right >
$$
The decomposition for the $\pi^- p$ and $\pi^0 n$ states are derived on page 7 of the lecture 9 slides.
For the other three possible $\pi N$ charge combinations (corresponding to the left hand side of the elastic scattering interactions listed above), write the description of the isospin state using the same nomenclature.  You may look up the Clebsch-Gordon coefficients in a Clebsch-Gordon table.

Write your answer here

### 2b.

Using the results from 2a. and the fact that the scatttering cross sections are proportional to $\left |H_{if} \right |^2$ calculate the ratios of the cross sections for the following processes:

\begin{eqnarray*}
(a) & & \frac{\sigma(\pi^{-} p\rightarrow \pi^{-} p)}{\sigma(\pi^{+} p\rightarrow \pi^{+} p)} \\
(b) & & \frac{\sigma(\pi^{-} p\rightarrow \pi^{0} n)}{\sigma(\pi^{-} p\rightarrow \pi^{-} p)}\\
(c) & & \frac{\sigma(\pi^{0} n\rightarrow \pi^{-} p)}{\sigma(\pi^{0} n\rightarrow \pi^{0} n)}
\end{eqnarray*}

Express your answers in terms of $M_{\frac{1}{2}}$ and $M_{\frac{3}{2}}$

Write your answer here

## Question 3: Isospin:  The $\Delta$ resonance (Thomson 9.3) (15 points)

### Learning objectives
In this question you will:
- Review the fact that isospin conservation in the strong interaction can also be used to relate decay rates
- Apply this concept to the case od $\Delta$ decays

By considering the isospin states show that the rates for the following strong interaction decays occur in the ratios:
$$
\begin{array}{cccc}
\Gamma(\Delta^{-} \rightarrow \pi^{-} n) : &
\Gamma(\Delta^{0} \rightarrow \pi^{-} p) : & 
\Gamma(\Delta^{0} \rightarrow \pi^{0} n) : & 
\Gamma(\Delta^{+} \rightarrow \pi^{+} n) : \\
 \Gamma(\Delta^{+} \rightarrow \pi^{0} p) : & 
\Gamma(\Delta^{++} \rightarrow \pi^{+} p)  & 
= &   3:1:2:1:2:3 &  \\
\end{array}
$$

Note:  Because the $\Delta$ can be created in $\pi^+ p$ scattering, we know that it has isospin $I=\frac{3}{2}$

Write your answer here

## Question 4 SU(3) $\lambda $ matrices (25 points)

### Learning objectives
In this question you will:
- Review the definition of the $\lambda$ matrices (the generators of SU(3) symmetry)
- Show that the SU(3) group contains three SU(2) subgroups and construct the raising and lowering operators for these subgroups

We learned in Quantum Mechanics that the smallest representation of angular momentum has dimension 2 (spin-$\frac{1}{2}$) and that the generators of infinitesmal rotations in this fundamental representation can be written as the three $2\times 2$ Pauli matrices $\sigma_x$, $\sigma_y$ and $\sigma_z$ that satisfy the SU(2) commutation relations 
$$\left [\frac{1}{2}\sigma_i, \frac{1}{2}\sigma_j \right ] = i \epsilon_{ijk} \frac{1}{2} \sigma_k$$

Similarly, the fundamental representation of SU3) can be written as a set of eight $3\times 3$ matrices called the $\lambda$-martices:

\begin{eqnarray*}
\lambda_1 = \left( 
\begin{array}{ccc} 0 & 1 & 0 \\ 1 & 0 & 0 \\ 0 & 0 & 0 \\
\end{array} \right)
& 
\lambda_2 = \left( 
\begin{array}{ccc} 0 & -i & 0 \\ i & 0 & 0 \\ 0 & 0 & 0 \\
\end{array} \right)
&
\lambda_3 = \left( 
\begin{array}{ccc} 1 & 0 & 0 \\ 0 & -1 & 0 \\ 0 & 0 & 0 \\
\end{array} \right)
\\
\lambda_4 = \left( 
\begin{array}{ccc} 0 & 0 & 1 \\ 0 & 0 & 0 \\ 1 & 0 & 0 \\ 
\end{array} \right)
& 
\lambda_5 = \left( 
\begin{array}{ccc} 0 & 0 & -i \\ 0 & 0 & 0 \\ i & 0 & 0 \\
\end{array} \right)
&
\lambda_6 = \left( 
\begin{array}{ccc} 0 & 0 & 0 \\ 0 & 0 & 1 \\ 0 & 1 & 0 \\
\end{array} \right)
\\
\lambda_7 = \left( 
\begin{array}{ccc} 0 & 0 & 0 \\ 0 & 0 & -i \\ 0 & i & 0 \\
\end{array} \right)
& &
\lambda_8 = \frac{1}{\sqrt{3}}\left( 
\begin{array}{ccc} 1 & 0 & 0 \\ 0 & 1 & 0 \\ 0 & 0 & -2 \\
\end{array} \right)
 \\
\end{eqnarray*}

### 4a.

Examining the first three $\lambda$ shows that they are just the Pauli matrices with zeros added in the third row and column.  Thus, these three matrices form a subgroup of SU(2) within the SU(3) group and obey the commutation relations we know from angular momentum.  This is one of three SU(2) subgroups within the SU(2) group.  We call them:
$$
\begin{array}{lccc}
{\rm Isospin} & \lambda_1 & \lambda_2 & \lambda_3 \\
{\rm U-spin:} &  \lambda_4 & \lambda_5& \frac{1}{2} \left (\sqrt{3}\lambda_8+\lambda_3\right )\\
{\rm V-spin:} &  \lambda_6 & \lambda_7& \frac{1}{2} \left (\sqrt{3}\lambda_8 -\lambda_3 \right )\\
\end{array}
$$
Using explicity matrix multiplication.  Show that the generator of U-spin and V-spin satisfy the commutation relations that define the SU(2) algebra.

Write your answer here

### 4b.

The fundamental representation of SU(3) can be written as the three column matrices which in we call in the quark model:
$$
\begin{array}{ccc}
u \equiv \left (
\begin{array}{c} 
1\\ 0 \\ 0 
\end{array}
\right )
&
d \equiv \left (
\begin{array}{c} 
0\\ 1 \\ 0 
\end{array}
\right )
& 
s \equiv \left (
\begin{array}{c} 
0\\ 0 \\ 1 
\end{array}
\right )
\\
\end{array}
$$
Show that the raising (lowering) operators
$$
\begin{array}{ccc}
I_{\pm} = \frac{1}{2} \left (\lambda_1 \pm i \lambda_2 \right ) &
U_{\pm} = \frac{1}{2} \left (\lambda_4 \pm i \lambda_5 \right )
& 
V_{\pm} = \frac{1}{2} \left (\lambda_6 \pm i \lambda_7 \right )
\end{array}
$$
turn $u$ into $d$ (and visa versa), $u$ into $s$ (and visa versa) and $d$ into $s$ (and visa veras)

Write your answer here

## Question 5: $\Lambda^{0} $ polarization in the strong interactions (20 points) 

### Learning objectives
In this question you will:

- Apply the concept of parity conservation in the strong interactions to determine what functional forms are allowed for a specific observable (the polarization)

Consider the production of $\Lambda$ particles via a strong interaction:

$$
p + p \rightarrow \Lambda^{0} + X
$$

where $X$ represents one of more additional final state particles that are not necessarily observed. Assume that the $\Lambda^{0}$ and  $X$ each have the same parity as the proton. Also assume that the initial protons are unpolarized.   In the center-of-mass frame, the momentum vectors of the two protons and the $\Lambda^{0}$ all lie in a plane, called the production plane:

<img src="productionPlane.png" alt="Drawing" style="width: 600px;"/>

It has been observed that $\Lambda^{0}$ (which are spin 1/2 particles) are polarized.  This means that the expectation value  $\left < \vec S_{\Lambda} \cdot \hat n\right > \ne 0$ for certain unit vectors $\hat n$.  Here $\vec S_{\Lambda}$ is the spin vector of the $\Lambda^{0}$.  

In this problem, you will determine what possible directions $\hat n$ can have.

### 5a. 

Show that if $\hat n$ lies in the production plane, then $\left < \vec S_{\Lambda} \cdot \hat n\right > = 0$ if parity is conserved 

#Write your answer here

### 5b. 

Show that a non-zero value of $\left < \vec S_{\Lambda} \cdot \hat n\right > $ is possible if $\hat n$ is perpendicular to the production plane

###  Hints:

1) Work in the center-of-mass frame since this is where the production plane is defined.  For each of the two cases (in the production plane and out of the production plane), the unit vector $\hat n$ direction can be written as an expression that depends the 3-momenta of the specified particles (perhaps with free parameters).

2) In the center of mass frame of two unpolarized colliding particles of the same species, the scattering rate must be be a symmetric function of $\cos \theta^*$ (the polar angle with respect to the $pp$ direction) since there is no distinction between the forward and backward directions.  


#Write your answer here